<a href="https://colab.research.google.com/github/Ahmed-L/CSE499A/blob/main/Dataset_499A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping North South University website

imports

In [1]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
import bs4
from html.parser import HTMLParser
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import numpy as np
import pandas as pd

python's HTMLParser

In [2]:
from html.entities import name2codepoint

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)
        for attr in attrs:
            print("     attr:", attr)

    def handle_endtag(self, tag):
        print("End tag  :", tag)

    def handle_data(self, data):
        print("Data     :", data)

    def handle_comment(self, data):
        print("Comment  :", data)

    def handle_entityref(self, name):
        c = chr(name2codepoint[name])
        print("Named ent:", c)

    def handle_charref(self, name):
        if name.startswith('x'):
            c = chr(int(name[1:], 16))
        else:
            c = chr(int(name))
        print("Num ent  :", c)

    def handle_decl(self, data):
        print("Decl     :", data)

parser = MyHTMLParser()

Demo run Using BS4 

In [3]:
document = requests.get('http://www.northsouth.edu/academic/sbe/')
soup = BeautifulSoup(document.text, "html.parser")
print(soup.find('title'))
print(soup.find_all('h1'))
print(soup.find_all('h2'))
print(soup.find_all('p'))

<title>School of Business &amp; Economics | North South University</title>
[]
[<h2 class="title">School of Business &amp; Economics</h2>]
[<p><img height="349" src="newassets/images/5-8746.AhW.PNG" width="285"/></p>, <p><strong>Welcome to NSU School of Business &amp; Economics !<br/></strong></p>, <p>We are unparalleled in the entire nation in terms of two of the most important enablers of business and economics education: faculty and facilities. I invite you to use this website to learn more about the qualifications and expertise our 100+ full-time faculty members in the four departments. Yes, you will discover that the core of our strength that drives the quality of our various undergraduate and graduate programs is our faculty, with all members having foreign degrees.</p>, <p>NSU Business School is located across four levels of North Academic Building at NSU’s gorgeous urban campus that is conveniently located in Bashundhara, Dhaka. I invite you to visit in person.</p>, <p>Our Schoo

Fetch all links from the base index page (Does not contain sub-page links)

In [4]:
index = 'http://www.northsouth.edu/'
document = requests.get(index)
links = []
soup = BeautifulSoup(document.text, "html.parser")

for link in soup.findAll('a'):
  var = link.get('href')
  if(var == '#' or var is None):
    continue
  else:
    if('http' in var):
      print(var)
      links.append(var)
    else:
      var = index + var
      links.append(var)
      print(var)


http://www.northsouth.edu/contact-us.html
http://www.northsouth.edu/archive.html
http://www.northsouth.edu/newsletter/nsu-catalog.html
http://www.northsouth.edu/journals/
http://www.northsouth.edu/newsletter.html
http://library.northsouth.edu
http://www.northsouth.edu/academic/academic-calendar/
http://institutions.northsouth.edu/oea/international-affairs/partnership/list-of-mous/
https://rds3.northsouth.edu/index.php/nsuresearch
http://www.northsouth.edu/
http://www.northsouth.edu/about/brief-history.html
http://www.northsouth.edu/about/vms.html
http://www.northsouth.edu/about/int-ab.html
http://www.northsouth.edu/about/accreditation.html
http://www.northsouth.edu/about/international-recognition.html
http://www.northsouth.edu/about/facts.html
http://www.northsouth.edu/about/bot/chairman-bot.html
http://www.northsouth.edu/about/bot/
http://www.northsouth.edu/about/bot/nsu-trust.html
http://www.northsouth.edu/about/executive-leaders.html#one
http://www.northsouth.edu/about/executive-lea

remove PDF links

In [5]:
for link in links:
  if('pdf' in link):
    links.remove(link)

for link in links:
  print(link)

http://www.northsouth.edu/contact-us.html
http://www.northsouth.edu/archive.html
http://www.northsouth.edu/newsletter/nsu-catalog.html
http://www.northsouth.edu/journals/
http://www.northsouth.edu/newsletter.html
http://library.northsouth.edu
http://www.northsouth.edu/academic/academic-calendar/
http://institutions.northsouth.edu/oea/international-affairs/partnership/list-of-mous/
https://rds3.northsouth.edu/index.php/nsuresearch
http://www.northsouth.edu/
http://www.northsouth.edu/about/brief-history.html
http://www.northsouth.edu/about/vms.html
http://www.northsouth.edu/about/int-ab.html
http://www.northsouth.edu/about/accreditation.html
http://www.northsouth.edu/about/international-recognition.html
http://www.northsouth.edu/about/facts.html
http://www.northsouth.edu/about/bot/chairman-bot.html
http://www.northsouth.edu/about/bot/
http://www.northsouth.edu/about/bot/nsu-trust.html
http://www.northsouth.edu/about/executive-leaders.html#one
http://www.northsouth.edu/about/executive-lea

In [6]:
title_list = []
text_list = []

def fetch(link):
  content = requests.get(link)
  #text_file = open("data.txt", "a")
  title_df = ""
  document = ""
  soup = BeautifulSoup(content.text, "html.parser")
  try:
    title = soup.find('title').text.strip()
    #text_file.write(str(title))
    title_df = str(title)
    print(title)
  except:
    print("*Title does not have any text in the html.*")
  try:
    for var in soup.findAll('h1'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      document += str(var.text.strip())

    for var in soup.findAll('h2'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      document += str(var.text.strip())

    for var in soup.findAll('h3'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      document += str(var.text.strip())

    for var in soup.findAll('h6'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      document += str(var.text.strip())

    for var in soup.findAll('p'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      document += str(var.text.strip())

  except:
      print("error in text extraction")
      pass

  title_list.append(title_df)
  text_list.append(document)
  #text_file.close()

fetch contents from links

In [7]:
#dataframe = pd.DataFrame(columns=['Title', 'Text'])

for link in links:
  dataframe = fetch(link)

Streaming output truncated to the last 5000 lines.
46
Rashenda Aziz
1231441080
47
Shawkat Hossain Swapan
1431217080
48
Nusrat Fatema Chowdhury
1530316080
49
Sadia Islam Swarna
1512956080
50
Md. Ridwan-UI- Rabbi
1513284080
51
Redwan Bin Abdul Baten
1513382080


52
Md. Ashraful Arif
1311328080
53
Mohammad Omar Faruque
1421854080
54
Sofia Salam
1321449080
55
Kakon Farjana
1421622080
56
Md. Salahuddin
1513186080
57
Syeda Hasina Ara Karim
072817080
58
Abu Kalam Md. Taifur Rahman
1512942080
59
Saeem Arafat Hossain
1421861080
60
Syed Nurul Aziz
1421862080
61
Sabbir Ahmed
1431447080
62
Sohana Sadique
1431446080
63
Sharmin Kauser
1411467080
64
Dilshad Afrin
1512907080
65
Anamika Haque
1311176080
66
Mst. Nurun Naher
1231385080
67
Samma Tabassum Ahmed
1431371080
68
Mahad Mohamed Rage
1611358080
69
Ahmed Nor Osman Hassan
1531302080
70
Merina Mustari
1321531080
71
Rezaur Rahman
082817080
72
Abdulahi Mohamed Matan
1612887080
73
K.M. Abdullah - Al - Masood
1531426080
74
Syeda Mushrefa Jahan
143127008

In [8]:
title = pd.DataFrame(title_list, columns=["Title"])
text = pd.DataFrame(text_list, columns=["Text"])

temp = pd.concat([title, text], axis = 1)
dataframe = pd.concat([dataframe, temp], axis = 0)

In [9]:
print(dataframe.head())

                                  Title  \
0     Contact  | North South University   
1      Archive | North South University   
2     Catalogs | North South University   
3     Journals | North South University   
4  Newsletters | North South University   

                                                Text  
0  ContactLocation MapBashundhara, Dhaka-1229, Ba...  
1  ArchiveBashundhara, Dhaka-1229, Bangladesh\n  ...  
2  CatalogsBashundhara, Dhaka-1229, Bangladesh\n ...  
3  JournalsNSBR Copy Right Agreement..NSBR Admini...  
4  NewslettersBashundhara, Dhaka-1229, Bangladesh...  


In [10]:
print(len(title_list))

160


In [11]:
dataframe.dropna(inplace=True)

dataframe.to_csv(r'dataframe.csv', index=False)